In [1]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
#cobra_model = load_json_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.json")



from cobra.io.mat import *
#from itertools import compress



cobra_model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")

import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo
    
def get_data_for_heatmap(explanatory_subgraph_path):
        
    explanatory_subgraph = pd.read_parquet(explanatory_subgraph_path).reset_index(drop=True)

    ray.shutdown()
    ray.init()
    @ray.remote


    def get_genes(node_1, node_2):
        
        if node_2 in dict_rxn_to_gene.keys():
            
            return dict_rxn_to_gene[node_2]
            
        elif node_1 in dict_rxn_to_gene.keys():
            
            return dict_rxn_to_gene[node_1]
            
        else:
            return ['']
        
    g = [] 
    for node_1, node_2 in zip(explanatory_subgraph.node1.tolist(), explanatory_subgraph.node2.tolist()):   
        g.append(get_genes.remote(node_1, node_2 ))

    
    genes = pd.Series(ray.get(g) , name='genes')



    assert genes.shape[0] == explanatory_subgraph.shape[0]


    return pd.concat([genes, explanatory_subgraph], axis=1)




to_R_heatmap_explanatory_subgraph_MASKED_GIN_Fluxes = get_data_for_heatmap("./results/explanations/Explanatory_subgraph_MASKED_GIN_Fluxes.parquet.gzip")
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concentration = get_data_for_heatmap("./results/explanations/Explanatory_subgraph_MASKED_GIN_Concentration.parquet.gzip")
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concen_plus_Fluxes = get_data_for_heatmap("./results/explanations/Explanatory_subgraph_MASKED_GIN_Concen_plus_Fluxes.parquet.gzip")

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x
2023-06-19 19:51:12,411	INFO worker.py:1518 -- Started a local Ray instance.
2023-06-19 19:51:27,976	INFO worker.py:1518 -- Started a local Ray instance.
2023-06-19 19:51:37,146	INFO worker.py:1518 -- Started a local Ray instance.


In [2]:
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Fluxes.reset_index(drop=True).to_csv(
    "./results/explanations/to_functional_R_heatmap_explanatory_subgraph_MASKED_GIN_Fluxes.csv")
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concentration.to_csv(
    "./results/explanations/to_functional_R_heatmap_explanatory_subgraph_MASKED_GIN_Concentration.csv")
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concen_plus_Fluxes.to_csv(
    "./results/explanations/to_functional_R_heatmap_explanatory_subgraph_MASKED_GIN_Concen_plus_Fluxes.csv")